
# RAG Demo

Este notebook demonstra um pipeline simples de **Retrieval-Augmented Generation (RAG)** usando:
- `sentence-transformers` para embeddings
- `faiss` (faiss-cpu) para índice vetorial
- `transformers` (ex.: `google/flan-t5-small`) como gerador

> Observação: execute este notebook no Google Colab (Runtime > Change runtime type > GPU) para melhor performance.
> As primeiras células instalam dependências. Se estiver usando Colab, deixe o `pip` rodar sem problemas.


In [ ]:

# Instala dependências (Colab precisa disto)
# Execute esta célula no Colab. Pode demorar alguns minutos dependendo da internet/CPU.
!pip install -q sentence-transformers faiss-cpu transformers[sentencepiece] torch scikit-learn matplotlib nbformat

# Verificar versões
import importlib, sys
import pkgutil
for pkg in ('sentence_transformers','faiss','transformers','torch','sklearn'):
    try:
        m = importlib.import_module(pkg if pkg!='sentence_transformers' else 'sentence_transformers')
        print(pkg, "importado com sucesso")
    except Exception as e:
        print("Erro importando", pkg, ":", e)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 37.1 MB/s eta 0:00:00
sentence_transformers importado com sucesso
faiss importado com sucesso
transformers importado com sucesso
torch importado com sucesso
sklearn importado com sucesso


## Pipeline: Embeddings → FAISS → Recuperação

In [ ]:

# Exemplo de fluxo RAG mínimo
# 1) Carregar uma pequena base de conhecimento (exemplos locais)
documents = [
    {"id": "doc1", "text": "Python é uma linguagem de programação de alto nível com tipagem dinâmica."},
    {"id": "doc2", "text": "FAISS é uma biblioteca para busca eficiente por similaridade em vetores densos."},
    {"id": "doc3", "text": "Hugging Face fornece a biblioteca transformers e um hub de modelos pré-treinados."},
    {"id": "doc4", "text": "Retrieval-Augmented Generation combina um recuperador e um gerador para respostas baseadas em KB."},
    {"id": "doc5", "text": "SentenceTransformers oferece modelos de embeddings de alta qualidade."},
]

# 2) Criar embeddings com sentence-transformers
from sentence_transformers import SentenceTransformer
embed_model_name = 'sentence-transformers/all-mpnet-base-v2'
embed_model = SentenceTransformer(embed_model_name)
texts = [d['text'] for d in documents]
embeddings = embed_model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
print('Embeddings shape:', embeddings.shape)

# 3) Construir índice FAISS
import faiss
import numpy as np
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings).astype('float32'))
print('Número de vetores no índice:', index.ntotal)

# 4) Recuperação: exemplo de query
query = "Para que serve FAISS?"
q_emb = embed_model.encode([query], convert_to_numpy=True)
k = 2
distances, indices = index.search(q_emb.astype('float32'), k)
retrieved = [documents[int(i)]['text'] for i in indices[0]]
print('\nDocumentos recuperados:')
for r in retrieved:
    print('-', r)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings shape: (5, 768)
Número de vetores no índice: 5

Documentos recuperados:
- FAISS é uma biblioteca para busca eficiente por similaridade em vetores densos.
- SentenceTransformers oferece modelos de embeddings de alta qualidade.


## Geração usando o contexto recuperado (RAG)

In [7]:

# 5) Geração condicional com contexto recuperado (RAG)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
gen_model_name = 'google/flan-t5-small'  # modelo pequeno suficiente para demo
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

context = '\n\n---\n\n'.join(retrieved)
prompt = f"""Use o contexto abaixo para responder a pergunta.

Contexto:
{context}

Pergunta: {query}

Resposta:"""

inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=1024)
out = gen_model.generate(**inputs, max_length=128)
answer = tokenizer.decode(out[0], skip_special_tokens=True)
print('\nResposta gerada:\n', answer)



Resposta gerada:
 For FAISS, what is the purpose of the text?


## Salvando resultados / Persistência

In [8]:

# 6) Salvar resultados (opcional) - você pode montar o Google Drive no Colab se quiser persistir
# Exemplo simples de salvar em arquivo local (no ambiente do Colab)
with open('rag_output_answer.txt', 'w', encoding='utf-8') as f:
    f.write('Query:\n' + query + '\n\nRetrieved:\n' + '\n'.join(retrieved) + '\n\nAnswer:\n' + answer)
print('Saída salva em rag_output_answer.txt')


Saída salva em rag_output_answer.txt



---

## Dicas para uso no Colab
- Altere Runtime → Change runtime type → GPU (se disponível) para acelerar geração e embeddings.  
- Se ocorrer `CUDA out of memory`, tente reduzir `max_length` ou use modelos ainda menores (ex.: `t5-small`).  
- Para projetos reais, troque FAISS local por um vector DB gerenciado (Milvus, Pinecone, Weaviate, etc.) se precisar de escala.